<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#GeoDataframe-/-Carto" data-toc-modified-id="GeoDataframe-/-Carto-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>GeoDataframe / Carto</a></span></li></ul></li><li><span><a href="#Descubrimos-en-clase-PyJsonViewer" data-toc-modified-id="Descubrimos-en-clase-PyJsonViewer-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Descubrimos en clase PyJsonViewer</a></span><ul class="toc-item"><li><span><a href="#Desde-archivo" data-toc-modified-id="Desde-archivo-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Desde archivo</a></span></li><li><span><a href="#Desde-el-propio-Jupyter" data-toc-modified-id="Desde-el-propio-Jupyter-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Desde el propio Jupyter</a></span></li></ul></li><li><span><a href="#Desgranamos-el-JSON-a-mano-después-de-haberlo-visto-bonito" data-toc-modified-id="Desgranamos-el-JSON-a-mano-después-de-haberlo-visto-bonito-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Desgranamos el JSON a mano después de haberlo visto bonito</a></span><ul class="toc-item"><li><span><a href="#Break----¿Dónde-están-entonces-mis-datos-finales?-(pensemos)" data-toc-modified-id="Break----¿Dónde-están-entonces-mis-datos-finales?-(pensemos)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Break -- ¿Dónde están entonces mis datos finales? (pensemos)</a></span></li><li><span><a href="#Probamos-la-funcion-getFromDict" data-toc-modified-id="Probamos-la-funcion-getFromDict-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Probamos la funcion getFromDict</a></span></li></ul></li><li><span><a href="#¿Lo-vemos-claro?" data-toc-modified-id="¿Lo-vemos-claro?-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>¿Lo vemos claro?</a></span></li><li><span><a href="#Exportamos-para-MongoDB" data-toc-modified-id="Exportamos-para-MongoDB-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Exportamos para MongoDB</a></span></li><li><span><a href="#El-Mapita-de-CartoFrames" data-toc-modified-id="El-Mapita-de-CartoFrames-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>El Mapita de CartoFrames</a></span></li></ul></div>

### GeoDataframe / Carto

![mapa](../images/geo_portada.jpg)

In [1]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

In [2]:
donde = "Puerta del Sol, Madrid"

Hay APIS para un montón de cosas, aquí os dejo GEOCODE
- https://geocode.xyz/

EJEMPLO PARA LLAMAR A LA API DE GEOCODE
```python
url_geocode = f"https://geocode.xyz/{DIRECCIÓN QUE QUIERO CONVERTIR}?json=1" # string con la ciudad
```

In [6]:
data = requests.get(f"https://geocode.xyz/{donde}?json=1").json()

In [7]:
# Vemos la response de Geocode
data

{'standard': {'addresst': 'Puerta Del Sol',
  'city': 'Madrid',
  'prov': 'ES',
  'countryname': 'Spain',
  'postal': {},
  'confidence': '1.00'},
 'longt': '-3.70343',
 'alt': {},
 'elevation': {},
 'latt': '40.41703'}

In [8]:
data["standard"]["addresst"]

'Puerta Del Sol'

In [9]:
data["longt"]

'-3.70343'

In [10]:
data["latt"]

'40.41703'

¿Qué es tipo point?
```python
coordenadas_lugar = {"type": "Point", "coordinates": [latitud, longitud]}
```

In [11]:
def geocode(direccion):
    """
    Esta función saca las coordenadas de la dirección que le pases
    """
    data = requests.get(f"https://geocode.xyz/{direccion}?json=1").json()
    try:
        return {"type": "Point", "coordinates": [data["latt"], data["longt"]]}
    except:
        return data

In [12]:
geocode("Paseo de la chopera 14, Madrid")

{'type': 'Point', 'coordinates': ['40.39652', '-3.70110']}

In [13]:
geocode("Paseo de la chopera 14")

{'error': {'description': 'Your request produced no suggestions.',
  'code': '018'}}

In [14]:
madrid = geocode("Madrid")

In [15]:
madrid

{'type': 'Point', 'coordinates': ['40.39818', '-3.62207']}

A veces Geocode Falla, así que plan es sacar las coordenadas de la web

In [ ]:
madrid_porsifalla = {'type': 'Point', 'coordinates': [-3.6793, 40.42955]}

¿Queremos hacernos una base de datos con todos los restaurantes Veganos de Madrid que hay en Foursquare? 
EEEEAAAASSSSYYYY (¡¡Somos programadoras!!) 🚀🙃🐭🔥🐼🤯

Lamento deciros que lo primerito es leer la documentación de la API. 
- https://developer.foursquare.com/docs/api-reference/venues/search/

#QUEYOYAESTABAREGISTRADA así que ya tengo token ....

In [ ]:
# Pero lo he guardado en un .env para que no me llegue ningún e-mail BOMBA 💌💣

In [16]:
load_dotenv()

True

In [17]:
client_id = os.getenv("tok1")
client_secret = os.getenv("tok2")

In [18]:
#url = "https://favqs.com/api/session"
url_query = 'https://api.foursquare.com/v2/venues/search'
url_recomendados = 'https://api.foursquare.com/v2/venues/explore'

Los parámetros de foursquare están explicados [aquí](https://developer.foursquare.com/docs/api-reference/venues/search/)

In [122]:
parametros = {
    "client_id": client_id,
    "client_secret": client_secret,
    "v": "20180323",
    "ll": f"{madrid['coordinates'][0]}, {madrid['coordinates'][1]}",
    "query": "Librería"
}

In [123]:
resp = requests.get(url_query, params = parametros).json()

In [124]:
type(resp)

dict

In [125]:
resp["response"].keys()

dict_keys(['venues'])

## Descubrimos en clase PyJsonViewer
Podemos ver los json de forma súper amigable

In [126]:
import pyjsonviewer

### Desde archivo
Exportamos el json con json.dump yo luego .view_data(json_file="ruta")

In [127]:
import json
with open('data.json', 'w') as f:
    json.dump(resp, f)

In [128]:
pyjsonviewer.view_data(json_file="data.json")

### Desde el propio Jupyter

In [77]:
pyjsonviewer.view_data(json_data=resp)

Hay que cerrar la interfaz de jsonviewer antes de abrir otro archivo, si no, da error.

## Desgranamos el JSON a mano después de haberlo visto bonito

In [78]:
#El value de la key groups del diccionario anterior está en tipo lista con un solo elemento
#Por eso nos quedamos con el elemento cero 👇🏻

Una vez explorado el json sé dónde están los ITEMS
```python
["response"]["groups"][0]["items"]
```

In [129]:
resp["response"]["venues"][0]["name"]

'Librería San Pablo'

In [130]:
resp["response"]["venues"][0]["location"]["lat"]

40.43609527176301

In [131]:
resp["response"]["venues"][0]["location"]["lng"]

-3.6421072483062744

In [132]:
# Otro truco para printear de forma amigable

In [133]:
json_form= json.dumps(resp, indent=2)
#print(json_form)

### Break -- ¿Dónde están entonces mis datos finales? (pensemos)

In [134]:
data = resp["response"]["venues"]

In [135]:
df = pd.DataFrame(data)

In [136]:
df.head()

,id,name,contact,location,categories,verified,stats,beenHere,hereNow,referralId,venueChains,hasPerk,venuePage
0,4ef07a168b81e3067f3368fd,Librería San Pablo,{},"{'address': 'C/ Alcalá, 387', 'lat': 40.436095...","[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636622434,[],False,NaN
1,51b18bec498ee6880f658439,Libreria San Pablo Jacinto Benavente,{},"{'address': 'Plaza Jacinto Benavente, 2', 'lat...","[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636622434,[],False,NaN
2,50b5d91be4b0311a46dc4e29,Libreria Jarcha,{},"{'address': 'Calle Lago Eire 6', 'lat': 40.404...","[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636622434,[],False,NaN
3,4bc9978f3740b71321f15e65,Librería Antonio Machado,{},"{'address': 'Fernando VI', 'lat': 40.424987845...","[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",False,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636622434,[],False,NaN
4,589851607b43b42ecd3f4f0d,Librería Balqís,{},"{'address': 'Calle de Alcalá 62', 'lat': 40.42...",[],True,"{'tipCount': 0, 'usersCount': 0, 'checkinsCoun...","{'count': 0, 'lastCheckinExpiredAt': 0, 'marke...","{'count': 0, 'summary': 'Nobody here', 'groups...",v-1636622434,[],False,{'id': '400677824'}


In [137]:
df.loc[1]["location"]

{'address': 'Plaza Jacinto Benavente, 2',
 'lat': 40.4400470485696,
 'lng': -3.637245048853146,
 'labeledLatLngs': [{'label': 'display',
   'lat': 40.4400470485696,
   'lng': -3.637245048853146}],
 'distance': 4834,
 'postalCode': '28012',
 'cc': 'ES',
 'city': 'Madrid',
 'state': 'Madrid',
 'country': 'España',
 'formattedAddress': ['Plaza Jacinto Benavente, 2',
  '28012 Madrid Madrid',
  'España']}

In [138]:
# Mis fuentes --stackoverflow

In [139]:
from functools import reduce
import operator

In [140]:
def getFromDict(diccionario,mapa):
    return reduce(operator.getitem,mapa,diccionario)

### Probamos la funcion getFromDict

In [141]:
resp["response"]["venues"][0]

{'id': '4ef07a168b81e3067f3368fd',
 'name': 'Librería San Pablo',
 'contact': {},
 'location': {'address': 'C/ Alcalá, 387',
  'lat': 40.43609527176301,
  'lng': -3.6421072483062744,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.43609527176301,
    'lng': -3.6421072483062744}],
  'distance': 4549,
  'postalCode': '28027',
  'cc': 'ES',
  'city': 'Madrid',
  'state': 'Madrid',
  'country': 'España',
  'formattedAddress': ['C/ Alcalá, 387', '28027 Madrid Madrid', 'España']},
 'categories': [{'id': '4bf58dd8d48988d114951735',
   'name': 'Bookstore',
   'pluralName': 'Bookstores',
   'shortName': 'Bookstore',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/bookstore_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
  'lastCheckinExpiredAt': 0,
  'marked': False,
  'unconfirmedCount': 0},
 'hereNow': {'count': 0, 'summary': 'Nobody

In [142]:
map_ = ["location", "lat"]

In [143]:
getFromDict(resp["response"]["venues"][0], map_)

40.43609527176301

In [144]:
resp["response"]["venues"][0]["location"]["address"]

'C/ Alcalá, 387'

In [145]:
def type_point(lista):
    return {"type":"Point", "coordinates": lista}

In [146]:
todo = {"nombre": ["name"], "dirección": ["location", "address"], "latitud": ["location", "lat"], "longitud": ["location", "lng"]}

In [148]:
librerias = resp["response"]["venues"] #Librerías es el json grande donde tengo toda la info de cada librería

In [149]:
type(librerias)

list

In [147]:
def extraetodo(json):
    todo = {"nombre": ["name"], "latitud": ["location", "lat"], "longitud": ["location", "lng"]} 
    total = []
    for elemento in json:
        libre = {key: getFromDict(elemento, value) for key,value in todo.items()}
        libre["location"] = type_point([libre["latitud"], libre["longitud"]])
        total.append(libre)
    return total

## ¿Lo vemos claro?

In [155]:
lovemosclaro = extraetodo(librerias) 
#lovemosclaro es la salida de la función extraetodo que ya es la lista de diccionarios limpios (con lo que quiero)

In [151]:
data = pd.DataFrame(lovemosclaro)

In [152]:
data.head()

,nombre,latitud,longitud,location
0,Librería San Pablo,40.436095,-3.642107,"{'type': 'Point', 'coordinates': [40.436095271..."
1,Libreria San Pablo Jacinto Benavente,40.440047,-3.637245,"{'type': 'Point', 'coordinates': [40.440047048..."
2,Libreria Jarcha,40.404841,-3.607384,"{'type': 'Point', 'coordinates': [40.404840764..."
3,Librería Antonio Machado,40.424988,-3.695981,"{'type': 'Point', 'coordinates': [40.424987845..."
4,Librería Balqís,40.421689,-3.682219,"{'type': 'Point', 'coordinates': [40.421689, -..."


In [103]:
cafes = data.copy()

In [69]:
libres = data.copy()

## Exportamos para MongoDB

In [154]:
import json
with open ("librerias.json","w") as f: # creamos un archivo vacío en el que vamos a escribir
    json.dump(lovemosclaro,f) # cargamos nuestra lista de diccionarios en ese archivo

In [156]:
type(f)

_io.TextIOWrapper

## El Mapita de CartoFrames

Para la columna geometry tenemos que poner en este orden Longitud, Latitud ⚠️🚨

In [ ]:
#!pip install cartoframes

In [105]:
import geopandas as gpd

In [ ]:
# Primero lo dataframeamos

In [111]:
type(libres)

pandas.core.frame.DataFrame

In [107]:
libres.sample()

,nombre,latitud,longitud,location
23,Librería Velázquez,40.526385,-3.652257,"{'type': 'Point', 'coordinates': [40.526384886..."


In [115]:
gdf = gpd.GeoDataFrame(libres, geometry=gpd.points_from_xy(libres.longitud, libres.latitud))

In [116]:
gdf.sample()

,nombre,latitud,longitud,location,geometry
8,Librería LaMalatesta,40.410809,-3.7032,"{'type': 'Point', 'coordinates': [40.410809, -...",POINT (-3.70320 40.41081)


In [117]:
type(gdf)

geopandas.geodataframe.GeoDataFrame

Al final hacemos un mapita con Cartoframes (quemegusta) y os dejo la DOC para que investiguéis porque no hay tiempo pa todo.
- https://carto.com/developers/cartoframes/

In [118]:
from cartoframes.viz import Map, Layer, popup_element

In [119]:
Map(Layer(gdf, "color:purple", popup_hover=[popup_element("nombre", "Librerías Chulas de Madrid")]))